In [1]:
import time
import random
import pygame

# initialize pygame, sound player
pygame.init()
pygame.mixer.init()


vol = 0.1          #set volume---------------------------------------------------------------------------------------------------------------------------------------------
ntrial = 1         #number of trials --------------------------------------------------------------------------------------------------------------------------------------
rb = 40            #radius of the ball-------------------------------------------------------------------------------------------------------------------------------------
app_dur = [3, 7]   #time for which ball is seen----------------------------------------------------------------------------------------------------------------------------
fl_dur = 0.15      #Flicker duration --------------------------------------------------------------------------------------------------------------------------------------
click_window = 1   #Time after flicker within which the subject must respond ----------------------------------------------------------------------------------------------
set_fps = 60       #Set fps of game----------------------------------------------------------------------------------------------------------------------------------------
rend_stats = True  #render fps and ime elapsed ----------------------------------------------------------------------------------------------------------------------------

# NOTE - if you change the fps, correspondingly change the speed. Higher the fps, lower the pixel speed you need to set to maintain same apparent velocity on screen.

# define width and height of screen----------------------------------------------------------------------------------------------------------------------------------------
width = 1600
height = 900
screen_res = (width, height)

right_ans = pygame.mixer.Sound("/home/yramakrishna/DeepLabCut/conda-environments/Sounds/reward.wav")
wrong_ans = pygame.mixer.Sound("/home/yramakrishna/DeepLabCut/conda-environments/Sounds/failure.wav")
right_ans.set_volume(vol)
wrong_ans.set_volume(vol)

clock = pygame.time.Clock()

pygame.display.set_caption("GFG Bouncing game")
screen = pygame.display.set_mode(screen_res)

correct_clicks = 0
wrong_clicks = 0
tooerc = 0
trial = 1

llb, lub = app_dur
# timec = clock.get_time()
timec = pygame.time.get_ticks()



while trial <= ntrial:
    pygame.event.clear()
    x = random.randint(300, width-300)
    y = random.randint(300, height-300)
    # Define the fixation point coordinates
    fixation = [x,y]
    fixation_x, fixation_y = fixation #randomize this
    fixation_radius = rb

    # Draw the fixation point
    fixation_color = (255, 255, 255)
    pygame.draw.circle(screen, fixation_color, (fixation_x, fixation_y), fixation_radius)

    # Update the display
    pygame.display.flip()

    # Wait for a mouse click on the fixation point to start the trial
    trial_started = False
    while not trial_started:
        for event in pygame.event.get():
            if event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
                trial_started = True


    # define colors
    red = (255, 0, 0)
    dred = (200, 0, 50)
    black = (0, 0, 0)

    # define ball
    ball_obj = pygame.draw.circle(
        surface=screen, color=red, center=fixation, radius=rb)
    
    start_pt = fixation
    
    # define speed of ball
    
    # speed = [X direction speed, Y direction speed]
    
    speed = [9, 9] #--------------------------------------------------------------------------------------------------------------------------------------------------------------

    limit = random.randint(llb, lub) 

    last_seen = []
    all_points = []

    start_time = time.time()
    flick_em1 = random.randint(0,1)
    flick_em = bool(flick_em1)
    flickr = limit-(fl_dur+click_window)
    bound = [start_time + flickr, start_time+flickr+fl_dur]
    
    stptx = min(start_pt[0], width-100)
    stpty = min(start_pt[1], height-100)
    
    random.seed(time.time())
    iwr = width #random.randint(stptx, width)
    iwl = 0 #random.randint(stptx, 0)
    iwd = height #random.randint(stpty, height)
    iwu = 0 #random.randint(stpty, 0)
    click = False #registering clicks
    tooer = False #registering early clicks
    
    # game loop
    while True:
        # event loop

        if (time.time() - start_time) >= limit:
            last_seen = ball_obj.center
            break

        # fill black color on screen
        screen.fill(black)

        # move the ball
        # Let center of the ball is (100,100) and the speed is (1,1)
        ball_obj1 = ball_obj.move(speed)
        # Now center of the ball is (101,101)
        # In this way our wall will move

        # if ball goes out of screen then change direction of movement --------------------------------------------------------------------------------------------------------
        if ball_obj.left <= 0 or ball_obj.right >= width:
            speed[0] = -speed[0]
        if ball_obj.top <= 0 or ball_obj.bottom >= height:
            speed[1] = -speed[1]
        
        # #ball changes movement randomly -------------------------------------------------------------------------------------------------------------------------------------
        # if ball_obj1.left <= max(iwl, 0): 
        #     speed[0] = -speed[0]
        #     iwr = random.randint(min(ball_obj.right + 300, width), width)
        # if ball_obj1.right >= min(iwr, width):
        #     speed[0] = -speed[0]
        #     iwl = random.randint(0, max(ball_obj.left - 300, 0))
        # if ball_obj1.top <= max(iwu, 0):
        #     speed[1] = -speed[1]
        #     iwd = random.randint(min(ball_obj.bottom + 300, height), height)
        # if ball_obj1.bottom >= min(height,iwd):
        #     speed[1] = -speed[1]
        #     iwu = random.randint(0, max(ball_obj.top - 300, 0))

        ball_obj = ball_obj.move(speed)

        if (time.time() >= bound[0] and time.time() <= bound[1]) and flick_em:
            pygame.draw.circle(surface=screen, color=dred, center=ball_obj.center, radius=rb)  
        # draw ball at new centers that are obtained after moving ball_obj
        else:
            pygame.draw.circle(surface=screen, color=red, center=ball_obj.center, radius=rb)
        
        
        timec = pygame.time.get_ticks()/1000
        clock.tick(set_fps)
        fps_disp = clock.get_fps()

        if rend_stats == True:
            font = pygame.font.SysFont('Arial', 10)
            text4 = font.render(f'Time taken: {timec}', True, (255, 255, 0))
            text5 = font.render(f'FPS: {fps_disp}', True, (255, 255, 0))
            screen.blit(text4, (0, 0))
            screen.blit(text5, (100, 0))

        # update screen
        pygame.display.flip()
        
        if time.time() <= (bound[0]):
            for event in pygame.event.get():
                    if event.type == pygame.KEYDOWN:
                        if event.key == pygame.K_SPACE:
                            tooer = True
            
        else:
            if click != True and (time.time() - start_time) <= limit:
                for event in pygame.event.get():
                    if event.type == pygame.KEYDOWN:
                        if event.key == pygame.K_SPACE:
                            click = True
        if tooer == True:
            break
    
    if tooer == True:
        tooerc += 1
        pygame.mixer.Sound.play(wrong_ans)
    else:
        trial = trial+1
        if click == flick_em:
            correct_clicks += 1
            pygame.mixer.Sound.play(right_ans)
        else:
            wrong_clicks += 1
            pygame.mixer.Sound.play(wrong_ans)
            
            
    for i in range(1, 1000):
        pygame.draw.circle(surface=screen, color=black, center=ball_obj.center, radius=rb)
        pygame.display.flip()

    
    print(flick_em, click)
    print("")
                    
for i in range(1, 1000): #displays results for a few seconds
    font = pygame.font.SysFont(None, 48)
    text = font.render(f"Correct guess = {correct_clicks}", True, (255, 255, 255))
    text2 = font.render(f"Wrong guess = {wrong_clicks}", True, (255, 255, 255))
    text3 = font.render(f" Too hasty = {tooerc}", True, (255, 255, 255))
    screen.blit(text, (width/2 - text.get_width()/2, height/2 - 3 * text.get_height()/2))
    screen.blit(text2, (width/2 - text2.get_width()/2, height/2))
    screen.blit(text3, (width/2 - text3.get_width()/2, height/2 + 3 * text3.get_height()/2))
    pygame.display.flip()

pygame.display.quit()
exit()

pygame 2.3.0 (SDL 2.24.2, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html
True True

